# Importing the required Libraries

In [1]:
from __future__ import print_function
from gensim.models import KeyedVectors
import pandas as pd
import numpy as np
from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import math
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

Using TensorFlow backend.


## Data Preprocessing

In [2]:
train_df = pd.read_csv("../input/questions/questions.csv")
train_df = train_df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6'],axis = 1)
train_df = train_df.dropna()
train_df.head(10)

,Category,Question
0,2.0,क्या बीजेपी को प्रियंका की टक्कर में वरुण गांध...
1,2.0,Narander modi ji ka speech aapko ksa lgta h ?
2,2.0,क्या ममता बनर्जी को धरने में बैठने से कोई फायद...
3,2.0,भारत की राजनीति में सबसे घटिया आदमी कौन है ?
4,2.0,अरविंद केजरीवाल को प्रधानमंत्री मोदी इतने बुरे...
5,2.0,पीएम मोदी राफेल सौदे से जुड़े विपक्ष के सवालों...
6,2.0,यदि सपा बसपा गठबंधन में कॉन्ग्रेस भी शामिल हो ...
7,2.0,हिंदू-मुसलमान विवाद से किसी देश को किस प्रकार ...
8,2.0,कौन सा भाषा है जो कि सब से ज्यादा देशों में बो...
9,2.0,गौरव जी आप यह बताइए कि आप ने बीजेपी की पार्टी ...


In [3]:
train_df_temp = pd.get_dummies(train_df['Category'])
train_df['Politics'] = train_df_temp[2.0]
train_df['Movies'] = train_df_temp[3.0]
train_df = train_df.drop(['Category'],axis = 1)

In [4]:
train_df = shuffle(train_df)
train_df

,Question,Politics,Movies
346,राजा हिंदुस्तानी की मूवी की हीरोइन का नाम? ?,0,1
119,क्या आरक्षण जाति आधार पर होना सही है ?,1,0
289,मध्यप्रदेश में किसकी बनेगी सरकार ?,1,0
27,भारत का युवा वर्ग आज किस दिशा की और जा रहा है ...,1,0
290,अगले लोकसभा चुनाव में बीजेपी कितनी सीट आएगी,1,0
157,पंजाब सरकार में मौजूदा मंत्री नवजोत सिंह सिद्ध...,1,0
262,राजस्थान में क्या कांग्रेसी आ पाएगी ?,1,0
205,मोदी जी के बाद अगला प्रधानमंत्री कौन रहेगा ?,1,0
277,इस बार राजस्थान में किसकी सरकार( 1)भाजपा (2)का...,1,0
137,आज भारत वर्ष का सबसे ज्वलंत मुद्दा क्या है,1,0


From the above data frame it is found that their are 368 questions

## Loading Hindi word embeddings from fastText

In [5]:
en_model = KeyedVectors.load_word2vec_format('../input/fasttext-hindi-300-vec/cc.hi.300.vec/cc.hi.300.vec')

emb = {}

for word in en_model.vocab:
    emb[word] = en_model[word]

/opt/conda/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


## Pre processing the text for input of the model

In [6]:
stringStop = "के का एक में की है यह और से हैं को पर इस होता कि जो कर मे गया करने किया लिये अपने ने बनी नहीं तो ही या एवं दिया हो इसका था द्वारा हुआ तक साथ करना वाले बाद लिए आप कुछ सकते किसी ये इसके सबसे इसमें थे दो होने वह वे करते बहुत कहा वर्ग कई करें होती अपनी उनके थी यदि हुई जा ना इसे कहते जब होते कोई हुए व न अभी जैसे सभी करता उनकी तरह उस आदि कुल एस रहा इसकी सकता रहे उनका इसी रखें अपना पे उसके \" \' [ ] . , ! : ; ?"
stopWords = stringStop.split(' ')
stopWords

['के',
 'का',
 'एक',
 'में',
 'की',
 'है',
 'यह',
 'और',
 'से',
 'हैं',
 'को',
 'पर',
 'इस',
 'होता',
 'कि',
 'जो',
 'कर',
 'मे',
 'गया',
 'करने',
 'किया',
 'लिये',
 'अपने',
 'ने',
 'बनी',
 'नहीं',
 'तो',
 'ही',
 'या',
 'एवं',
 'दिया',
 'हो',
 'इसका',
 'था',
 'द्वारा',
 'हुआ',
 'तक',
 'साथ',
 'करना',
 'वाले',
 'बाद',
 'लिए',
 'आप',
 'कुछ',
 'सकते',
 'किसी',
 'ये',
 'इसके',
 'सबसे',
 'इसमें',
 'थे',
 'दो',
 'होने',
 'वह',
 'वे',
 'करते',
 'बहुत',
 'कहा',
 'वर्ग',
 'कई',
 'करें',
 'होती',
 'अपनी',
 'उनके',
 'थी',
 'यदि',
 'हुई',
 'जा',
 'ना',
 'इसे',
 'कहते',
 'जब',
 'होते',
 'कोई',
 'हुए',
 'व',
 'न',
 'अभी',
 'जैसे',
 'सभी',
 'करता',
 'उनकी',
 'तरह',
 'उस',
 'आदि',
 'कुल',
 'एस',
 'रहा',
 'इसकी',
 'सकता',
 'रहे',
 'उनका',
 'इसी',
 'रखें',
 'अपना',
 'पे',
 'उसके',
 '"',
 "'",
 '[',
 ']',
 '.',
 ',',
 '!',
 ':',
 ';',
 '?']

In [7]:
print(train_df.Question.str.split().str.len().describe())
SEQ_LEN = 80 # Lenth of all questions are less than or equal to 67 thus 80 is taken as the length of input to the model

count    368.000000
mean      14.149457
std        8.812732
min        4.000000
25%        9.000000
50%       12.000000
75%       17.000000
max       67.000000
Name: Question, dtype: float64


In [8]:
import re
_WORD_SPLIT = re.compile("([.,!?\"':;)(])")
_DIGIT_RE = re.compile(br"\d")

def basic_tokenizer(sentence):
    """Very basic tokenizer: split the sentence into a list of tokens."""
    words = []
    for space_separated_fragment in sentence.strip().split():
        words.extend(_WORD_SPLIT.split(space_separated_fragment))
        
    return [w.lower() for w in words if w not in stopWords and w != '' and w != ' ']

In [9]:
def text_to_array(text):
    empyt_emb = np.zeros(300)
    text = basic_tokenizer(text[:-1])[:SEQ_LEN]
    embeds = [emb.get(x, empyt_emb) for x in text]
    embeds += [empyt_emb] * (SEQ_LEN - len(embeds))
    arr =  np.array(embeds)
    return arr

In [10]:
batch_size = 32

def batch_gen(train_df):
    n_batches = math.ceil(len(train_df) / batch_size)
    while True: 
        train_df = train_df.sample(frac=1.)  # Shuffle the data.
        for i in range(n_batches):
            texts = train_df.iloc[i*batch_size:(i+1)*batch_size, 0]
            text_arr = np.array([text_to_array(text) for text in texts])
            yield text_arr, np.array([train_df["Politics"][i*batch_size:(i+1)*batch_size],train_df["Movies"][i*batch_size:(i+1)*batch_size]]).transpose()

In [11]:
from keras.models import Sequential,Model
from keras.layers import LSTM, Flatten, Dense, Bidirectional,Input,Dropout

from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers,regularizers, constraints

In [12]:
model = Sequential()
model.add(Bidirectional(LSTM(10, return_sequences = True),input_shape = (SEQ_LEN,300)))
model.add(Bidirectional(LSTM(4)))
model.add(Dense(8,activation = 'relu'))
model.add(Dense(2,activation = 'softmax'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [13]:
mg = batch_gen(train_df)
model.fit_generator(mg, epochs=5,
                    steps_per_epoch=32,
                    verbose=True)

Epoch 1/5
32/32 [==============================] - 18s 566ms/step - loss: 0.5970 - acc: 0.8038
Epoch 2/5
32/32 [==============================] - 14s 451ms/step - loss: 0.4422 - acc: 0.8165
Epoch 3/5
32/32 [==============================] - 14s 443ms/step - loss: 0.3293 - acc: 0.8417
Epoch 4/5
32/32 [==============================] - 14s 445ms/step - loss: 0.1590 - acc: 0.9649
Epoch 5/5
32/32 [==============================] - 14s 446ms/step - loss: 0.1038 - acc: 0.9726


In [14]:
model.save('model.h5')

In [15]:
batch_size = 32

def batch_gen_test(texts):
    text_arr = np.array([text_to_array(text) for text in texts])
    yield text_arr

In [16]:
testQ = ['का मोदी सरकार भारत की संवैधािनक संसाओ','पाथेर पाचाली फिल्म को विदेशी पुरस्कार से कब सम्मानित किया गया','इस बार कौन सी फिल्म हिट होगी ','राष्ट्रीय स्वयंसेवक संघ का मुख्यालय कहाँ है ?','टाइगर श्रॉफ की पहली फिल्म का क्या नाम है','फिल्म का क्या नाम है']
test_questions = batch_gen_test(testQ)

In [17]:
for ques in test_questions:
    pred = model.predict(ques)
    print(pred)

[[0.9864314  0.01356859]
 [0.18723032 0.81276965]
 [0.15821949 0.84178054]
 [0.98491806 0.01508192]
 [0.15347007 0.84652996]
 [0.20394443 0.7960556 ]]


In [18]:
i = 0

for prediction in pred:
    print("Question : ",testQ[i])
    i = i+1
    if(prediction[0] >= 0.5):
        print("Question belongs to Political Category")
        print("Confidence : ",prediction[0]*100,"%")
    else:
        print("Question belongs to Movies Category")
        print("Confidence : ",prediction[1]*100,"%")

Question :  का मोदी सरकार भारत की संवैधािनक संसाओ
Question belongs to Political Category
Confidence :  98.64314198493958 %
Question :  पाथेर पाचाली फिल्म को विदेशी पुरस्कार से कब सम्मानित किया गया
Question belongs to Movies Category
Confidence :  81.27696514129639 %
Question :  इस बार कौन सी फिल्म हिट होगी 
Question belongs to Movies Category
Confidence :  84.17805433273315 %
Question :  राष्ट्रीय स्वयंसेवक संघ का मुख्यालय कहाँ है ?
Question belongs to Political Category
Confidence :  98.49180579185486 %
Question :  टाइगर श्रॉफ की पहली फिल्म का क्या नाम है
Question belongs to Movies Category
Confidence :  84.65299606323242 %
Question :  फिल्म का क्या नाम है
Question belongs to Movies Category
Confidence :  79.60556149482727 %
